# Delay Tolerant Networks

### Description

In several instances, network communications can occur with relevant delays, as for extra-terrestial communications or for supervised monitoring networks. In such network deciding which node can communicate with which other node and when is extremely complicated issue (a np-completed scheduling problem).

### Assignments

1. Generate a series of patterns of transmissions on a network represented as a graph. In such graph the edges are weighted accordingly to the delay between transmittion and reception.
2. Implement euristic algorithms to lower the potential of the network, i.e. aiming at minimizing the distance of the communications from their final destination.
3. Find out the most efficient algorithm

### Contacts
Leonardo Badia <badia@dei.unipd.it>

# Introduction to Delay Tolerant Networks

Delay Tolerant Networking (DTN) is a relatively new area of research which 
is rapidly becoming the focus of considerable effort. DTN is an approach
to computer architecture that seeks to address the technical issues in 
heterogeneus networks that may lack continuos network connectivity. The
final aim of this developing study area is to tackle problems which the 
Internet is not able to face.  

There's a great variety among the different DTNs: they range from deep 
space networks to buses driving around mobile wireless hotspots in 
villages to provide Internet service to kiosks. 

The big difference from the regular Internet Protocol (IP) is that IP
assumes a seamless end-to-end data path instead DTNs are built to account
for errors and disconnections. Unlike IP, where heterogeneus networks are 
connected by assigning each node a common network-layer identifier (IP address),
a DTN uses "naming, layering, encapsulation and persistent storage".

The situations in which we use DTNs are pretty challenging:

* The throughput is generally low (we can think for instance about the acoustic in underwater modems) and may 
be highly asymmetric or in the worse scenario a return channel might not 
exist

* These networks are frequently disconnected, not only by faults but due
to motion for instance

* These systems frequently operate with a low duty cycle and the queuing 
time can be very large; with these conditions TCP would time out the session
instead UDP would lose all the packets.

We decided to use 2 different approaches in order to face to routing problem with DTNs. The first one is a greedy
algorithm while the second is an epidemic algorithm. 

We decided to model the DTN as a wireless network where at each time step a link has a certain probability of being shut down for a certain amount of time. 

Before going into the details of the two different algorithms there are some assumptions that need to be pointed out:

* We are working with infinite buffers 

* There are no errors in transmission

* There's a maximum distance between two satellites beyond which they can't communicate

* We keep the same size for every packet and the same bandwidth for each link

* This is a centralized system, in the sense that Earth knows all the topology of the network

The simulated network is a satellitar network where the nodes need to send data to the main station on Earth, for the sake of semplicity all of them have a direct link to the the destination. This is not an oversemplification because given the fact that links often shut down the overall behaviour is similar to the one we would have in case of indirect communications.

We repeated the tests done below also for the latter case but they were not kept. This is due to the fact that since some satellites are too far to directly communicate with the base station it is necessary to increase the number of nodes in order to have a meaningful configuration.
Increasing the total number of nodes is prohibitive because both the algorithms are superlinear in the number of nodes, meaning that it takes too much time to execute them in this particular case.
In most cases the results were very similar to the one obtained by the simulation, if not it will be specified below.

In [ ]:
#used to import the utility functions
import sys
sys.path.insert(1, "utility_functions/")

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
import matplotlib.animation as animation
import math
import pylab as pl
from matplotlib import collections  as mc
import mpl_toolkits.mplot3d.axes3d as p3
from network_utilities import generate_network, plot_network, disable_links, adjacency_matrix
from greedy_routing import greedy_routing
from graphics import pkt_plot,plot_idle
from metrics import throughput, delay, deviations
from animation_utilities import get_links, check_active, update_lines, get_vel, update_graph
import pandas as pd
from epidemics import epidemic, remove_duplicates
from tqdm.notebook import tqdm
import copy

#MAIN PARAMETERS
################################################
n = 10                                         #
c = 3e8                                        #
T_tx = 10  # da regolare                       #
r_max = 60e9  # Marte                          #
tau_max = 200                                  #
delta_time = 10                                #
n_configurations = 10                          #
network_seed = 17363720                        #
################################################

In [ ]:
nodes = generate_network(n,r_max,seed=17363720)
A = adjacency_matrix(nodes,tau_max)
sorted_nodes = np.argsort(A[0,:])     #sorting based on distance from Earth
A = A[sorted_nodes,:][:,sorted_nodes]
nodes = nodes[sorted_nodes]

In [ ]:
%matplotlib notebook

bg = [26/255,26/255,26/255]
fig,ax = plot_network(nodes,sphere=True,r_max=r_max)
ax.set_facecolor(bg)
rotation = True

def rotate(num,ax,rotation):
        ax.azim = 0.5*num*rotation

ax.set_axis_off()
ani = animation.FuncAnimation(fig, rotate, interval=20, fargs=(ax,rotation))
plt.show()

# Greedy Routing

In every network where data is generated from a node and needs to be delivered to a different one, may it be the postal network or the internet, it is necessary to find out the route that the data should follow in order to reach its destination.

This is usually done by a routing algorithm, in the internet the most widely used ones are the Dijikstra and the Bellman-Ford algorithms, these two are used in order to find the shortest path from the source to the destination according to some metric that could be the number of hops, the end to end delay etc.

<!-- The code below is html and I used it because it gives much more freedom for
controlling images-->
<figure>
    <img src="figures/routing.svg" width=500/>
    <div style="text-align: center;"> 
        <i>
            Path obtained by applying routing between the two nodes
        </i>
    </div>
</figure>

Classical routing algorithms do not work on DTNs (Delay Tolerant Networks) because they were thought for systems where links are supposed to fail very rarely and where the end to end delay is relatively small.

In the internet the chance of a link failing is low and even if it does the end to end delay is small, so in case a packet is lost because of link failure sending the packet again is not too expensive.
In DTNs this is not true because:

* The propagation time is very high, expecially if we are considering satellite communication

* The chance of link failure is very high.

On top of that it is not even possible to find the best routing algorithm for networks, this is because if also the traffic generated by other nodes is taken into consideration the problem becomes NP.
By analyzing the state of the art in DTN it is possible to see that the best results are mostly obtained by using greedy algorithms, i. e. algorithms that take a decision by choosing the best choice at each iteration without considering long term planning.

The main underliying assumptions used in the implementation of the greedy algorithm are:
* Full knowledge of the topology and evolution of the network is given. This is not a very strict requirement as it might happen in real life, for example for satellite networks, where usually link failure is caused by the interference of a celestial body, the requirement can be easily satisfied.
* One central station (Earth), that fully knows the topology evolution of the network, calculates all the optimal routes beforehand and communicates them to each node. The fact that a central node perfoms all the computations allows for accurate scheduling that avoids collisions so no method has to be emplyed in order to detect them.

The implemented greedy algorithm chooses the best next hop by using a slightly modified version of the dijkstra algorithm that allows to compute the best path from the source to the destination at a specific point in time.

Since it is also necessary to take collisions into account, if sending one packet causes one, the sender needs to wait until it is sure that no collision will occur. After the idle time waited in order to avoid collisions the optimal path needs to be recomputed because, due to topology changes, it might not be optimal anymore.

## Modified Dijkstra

In general the Dijkstra algorithm is used to find the best path between two nodes in weighted graphs with positive weights.

<figure>
    <img src="figures/dijkstra.gif" width=300/>
    <div style="text-align: center;"> 
        <i>
            Dijkstra algorithm in action
        </i>
    </div>
</figure>

Usually the algorithm is run on graphs that have fixed topology, unfortunately for DNTs this is not the case, hence a link that was active when the packet was sent might not be active when the packet reaches its destination.

This has to be taken into account because a link failing while a message is being sent through the link will result in packet loss.

An implementation of the Dijkstra algorithm for DTNs that solved this problem was proposed in [1].
With this approach the weight of a node is considered equal to $t_p + t_{tr} + t_w$ where $t_p$ is the propagation time, $t_{tr}$ is the transmission time and $t_w$ is the total time that the sender needs to wait before sending because of link failures.

The pseudo code for the algorithm is:
```
Input = G=(E, V), s, T, w(e,t)
Q = V
L[s]=0
L[v]=inf for every v != s
while Q != {} do
    u = u in Q that minimizes L[u]
    Q = Q - u
    for each edge e in E between u and another vertex v do
        if L[v] > L[u] + w(e, T + L[u]) then
            L[v] = L[u] + w(e, T + L[u])
        end if
    end for
end while
```
Where G is the graph representing the network, s is the starting node, T is the starting time, w(e,t) is the weight of edge e at time t.
The only difference from the standard algorithm is that in the standard case w(e,t) is constant in t, meaning that by correctly computing the function w(e,t) the algorithm will be able to find shortest paths also in the case of changing topology.


## Algorithmic Analysis

In order to get a basic understanding on how the algorithm is working we decided to draw the following plots:

1) The first plot represents the number of packets that passed through a certain node during their trip from source to destination. The nodes are indicated as a number and they are ordered from the closest (node 1) to the farthest from Earth (node 9).

2) The second plot shows the occupation of the Earth, in particular the black lines represent moments when the Earth is not receiving any data, while the colored lines identify moments when Earth is receiving a packet. The color associates the packet to its source.

In [ ]:
#generating some random packets with earth as destination
packets = np.random.randint(1, n, (1000, 2))
packets[:,1] = 0

ttr = T_tx
At = disable_links(A, 100000, delta_time ,mode="unstable")
#performing the routing algorithm
results = greedy_routing(At, packets, ttr, delta_time, with_tqdm =True)

#finding out how many packets transited in one node
packet_distribution = np.zeros(n)

#for each packet
for packet in results:
    #for each node in the route
    for node in packet.route:
        #if the node is not the destination
        if node != packet.destination:
            packet_distribution[node] += 1
            
#keep copies of the original data for later
initial_nodes = np.copy(nodes)
initial_A = np.copy(A)
packets_greedy = copy.deepcopy(results) 

In [ ]:
%matplotlib inline
_, axs = plt.subplots(1, 2, figsize = (15, 5))
axs[0].bar(np.arange(n), packet_distribution)
axs[0].set_title("Number of packets that passed through a node")
axs[0].set_xlabel("Nodes id ordered by distance from earth")
axs[0].set_ylabel("# of packets")
axs[0].set_xticks(np.arange(n))
plot_idle(results, ttr, axs[1]) 

1) From the first plot it is possible to immediately notice an intuitive fact that is: 
 the closer a node is to the destination the more packets will pass through it. Since this was the expected behaviour it reassures us about the fact that the greedy algorithm is working properly because each node is trying to bring the packets closer to Earth.
The high number of packets passing through the 8th node is due to packets being redirected back because of links teardown, it is possible to visualize this phenomenon in the animation.

2) The key observation for the second plot is that Earth is almost always receiving packets, this means that the algorithm is very efficient.

### Transmission Time effect on the algorithm

In next section the outcome of the algorithm is tested in case the transmission time greatly changes, this value is related to the dimension of the packet (L) and to the bandwidth of the channel (B), in particular $t_{tr} = \frac{L}{B}$.
This helps us understand how the algorithm answers to low bandwidth networks or when it is necessary to send very big packets.

The following plots show how the arrival time, the throughput and the number of optimal routes changes as the transmission time increases.

In order to reduce the effect of noise we ran the algorithm multiple times with different network configurations and we averaged the outcome. The whole procedure was repeated with two different link deactivation probabilities in order to see how this impacts on the algorithm.

In [ ]:
np.random.seed(1000)
fig = plt.figure(figsize=(10, 10))

#generating the axes as wanted
gs = fig.add_gridspec(2, 2)
axs = []
axs.append(fig.add_subplot(gs[0, :]))
axs.append(fig.add_subplot(gs[1, 0]))
axs.append(fig.add_subplot(gs[1, 1]))

#generating some random packets with earth as destination
packets = np.random.randint(1, n, (100, 2))
packets[:,1] = 0

#trying out the outcomes with two different link deactivation modes
for mode in ["light", "unstable"]:

    #choosing the color to be used in the plot
    color = ["blue", "red"]
    color = color[mode == "light"]

    #defining the transmission times that will be tested
    ttrs = range(1, 200, 20)

    #outputs of the routing algorithm for each configuration and ttr
    outputs = np.zeros((len(ttrs), n_configurations))
    deviation_arr = np.zeros((len(ttrs), n_configurations))

    for rand_iter in tqdm(range(n_configurations)):

        #generate a new configuration
        nodes = generate_network(n, r_max)
        A = adjacency_matrix(nodes,tau_max)
        At = disable_links(A,500000, delta_time, mode=mode)

        #for each transmission time
        for ttr_ind in range(len(ttrs)):

            #calculate the route of each packet
            results = greedy_routing(At, packets, ttrs[ttr_ind], delta_time)
            #finding out the arrival times of each packet
            arrival_times = np.array([packet.arrival_times[-1] for packet in results])
            #the overall transmission time is the maximum of the arrival times
            outputs[ttr_ind, rand_iter] = np.max(arrival_times)
            #getting the number of packets that followed a route different from the ideal one
            deviation_arr[ttr_ind, rand_iter] = deviations(A, results)

    #finding the mean over the different configurations of the arrival times
    arrival_times = np.mean(outputs, axis = 1)

    #plotting
    axs[0].plot(ttrs, arrival_times, linestyle = "-", c = color)
    axs[0].plot(ttrs, arrival_times - packets.shape[0] * np.array(ttrs), linestyle = "-.", c = color)
    axs[1].plot(ttrs, throughput(packets.shape[0], arrival_times) * np.array(ttrs), linestyle = "--", c = color)
    axs[2].plot(ttrs, np.mean(deviation_arr, axis = 1), linestyle="--", c = color)

axs[0].plot(ttrs, packets.shape[0] * np.array(ttrs), linestyle = "--", c="black")
axs[0].set_ylabel("Time(s)")
axs[0].set_xlabel("Transmission Time(s)")
axs[0].set_title("Mean arrival time vs mean idle time")

axs[0].legend([
    "Arrival time(light)", 
    "Idle time(light)",
    "Arrival time(unst)",
    "Idle time(unst)",
    "Total Transmission Time",
])

axs[1].set_title("Throughput of the system")
axs[1].set_ylabel("Normalized Throughput")
axs[1].set_xlabel("Transmission Time(s)")
axs[1].legend([
    "Throughput(light)",
    "Throughput(unstable)"
])

axs[2].set_title("Percentage of packets that followed unoptimal route")
axs[2].set_ylabel("Percentage")
axs[2].set_xlabel("Transmission Time(s)")
axs[2].legend([
    "Percentage(light)",
    "Percentage(unstable)"
])
plt.tight_layout()

### Some insights on the plots

1) What we get in the first plot is that the idle time remains almost equal to zero even if we increase the transmission time a 
lot. The total transmission time can be expressed as $T_{tot} = t_{tx}n_{packets}$ and is the lower bound for the arrival time $t_a = T_{tot} + t_{idle}$. If we consider the light and unstable mode we can see that the results are very similar, the arrival time though is always higher for the unstable mode as expected; this is because if more links are being shut down it is reasonable to expect a longer arrival time.

2) With the second plot it is possible to see that for small values of the transmission time the normalized throughput is very small, this is due to the fact that the arrival time is dominated by the propagation time.
In other words since $t_{tx}$ is small it means that the Earth has a lot of idle time, meaning that the normalized throughput is not too high.
With $t_{tx}>50s$ we can see that the throughput is basically unaffected by the transmission time.
Also in this case as expected the light mode curve is above the unstable mode one.


3) In the third plot we can observe something very intuitive: the bigger the transmission time, the 
higher the percentage of non optimal routes will be; this is due to the fact that as $t_{tx}$ increase so does the queueing time, meaning that it is more likely that the path will be changed.
Here instead we can spot a big difference between light and unstable mode, given by the 
fact that with unstable mode we are shutting down many more links than with light mode so the number of unoptimal routes must be higher.

### Propagation time effects
The next plots investigate how the arrival time, throughput and percentage of optimal routes changes as functions of the distance between nodes. The distance between nodes is actually proportional to the propagation time $t_p = \frac{dist}{c}$ where c is the speed of light so the two analisys will yield the same result.

In [ ]:
ttr = T_tx
fig = plt.figure(figsize=(10, 10))

#generating the axes as wanted
gs = fig.add_gridspec(2, 2)
axs = []
axs.append(fig.add_subplot(gs[0, :]))
axs.append(fig.add_subplot(gs[1, 0]))
axs.append(fig.add_subplot(gs[1, 1]))

#iterating over two different modes
for mode in ["light", "unstable"]:

    color = ["blue", "red"]
    color = color[mode == "light"]

    #constants that multiplied by the propagation times simulate the expansion of the configuration
    r_maxes = np.array([2**((i-4)/2) for i in range(10)])

    #temporary variables used to store 
    outputs = np.zeros((len(r_maxes), n_configurations))
    deviation_arr = np.zeros((len(r_maxes), n_configurations))

    #for each different configuration
    for rand_iter in tqdm(range(n_configurations)):

        #generating a configuration
        nodes = generate_network(n, r_max)
        A = adjacency_matrix(nodes, tau_max)
        At = disable_links(A,200000, delta_time, mode=mode)

        #for each constant
        for i in range(len(r_maxes)):

            #keeping the same configuration but rescaling the proagation times
            results = greedy_routing(At*r_maxes[i], packets, ttr, delta_time)

            #getting the overall transmission time and the number of unoptimal routes
            arrival_times = np.array([packet.arrival_times[-1] for packet in results])
            outputs[i, rand_iter] = np.max(arrival_times)
            deviation_arr[i, rand_iter] = deviations(A*r_maxes[i], results)

    arrival_times = np.mean(outputs, axis = 1)
    axs[0].semilogx(r_maxes * r_max, arrival_times, linestyle = "-", c = color)
    axs[0].semilogx(r_maxes * r_max, arrival_times - packets.shape[0] * ttr, linestyle = "-.", c = color)
    axs[1].semilogx(r_maxes * r_max, throughput(packets.shape[0], arrival_times), linestyle="--", c = color)
    axs[2].plot(r_maxes * r_max, np.mean(deviation_arr, axis = 1), linestyle = "--", c = color)

axs[0].semilogx(r_maxes * r_max, np.ones(len(r_maxes)) * packets.shape[0] * ttr, linestyle = "--", c = "black")
axs[0].set_ylabel("Time(s)")
axs[0].set_xlabel("Radius")
axs[0].set_title("Mean arrival time vs mean idle time")
axs[0].legend([
    "Arrival time(light)", 
    "Idle time(light)",
    "Arrival time(unst)",
    "Idle time(unst)",
    "Total Transmission Time"
])

axs[1].set_title("Throughput of the system")
axs[1].set_ylabel("Normalized Throughput")
axs[1].set_xlabel("Radius")
axs[1].legend([
    "Throughput(light)", 
    "Throughput(unstable)",
])

axs[2].set_title("Percentage of packets that followed unoptimal route")
axs[2].set_ylabel("Percentage")
axs[2].set_xlabel("Radius")
axs[2].legend([
    "Percentage(light)",
    "Percentage(unstable)"
])
plt.tight_layout()    



### Some insights on the plots 

1) With the first plot we are visualizing the arrival time, the idle time and the total transmission 
time as a function of the maximum radius under light and unstable mode.
What we see is that:
      
* Either with light and unstable mode the idle time is almost constant up to a certain radius. This
    constant is close to zero for light mode whereas for unstable mode it is about 300 seconds. Then from 
    a certain radius on we see an increse which for light mode is very slow and instead for the unstable 
    mode is very fast but in both case the trend is linear with the propagation time.
    The fact that at the beginning the arrival time is almost constant is due to the fact that the 
    transmission time dominates the transmission while after $t_{tx}$ becomes negligible with respect to
    $t_p$ then the arrival time starts to become proportional with the maximum radius.


2) With the second plot we're pointing out the throughput as a function of the maximum radius;
what we observe is that the behaviour under light and unstable mode is similar but of course light mode gives us a higher throughput. Anyway for both modes the throughput is almost constant (for the unstable mode the constant is centered at 0.08 instead for the light mode it is centered at 0.09) like before when the transmission time becomes negligible with respect to the propagation time the transmission becomes dominated by the latter resulting in a lower normalized throughput.
  

3) In the last image we can observe the percentage of packets that followed unoptimal routes as a function of the maximum
radius. What we get for the two different modes is a similar behaviour and they don't express a clear relation with the maximum radius as it looks like it is more or less constant.

### Effect of the number of packets
Finally with the next plots we want to see how the various metrics change as the number of packets, and thus the network load, increases.

In [ ]:
fig = plt.figure(figsize=(10, 10))
#generating the axes as wanted
gs = fig.add_gridspec(2, 2)
axs = []
axs.append(fig.add_subplot(gs[0, :]))
axs.append(fig.add_subplot(gs[1, 0]))
axs.append(fig.add_subplot(gs[1, 1]))
ttr = T_tx

for mode in ["light", "unstable"]:

    color = ["blue", "red"]
    color = color[mode == "light"]

    #changing the number of packets
    packet_numbers = [2**i for i in range(1,13)]

    outputs = np.zeros((len(packet_numbers), n_configurations))
    deviation_arr = np.zeros((len(packet_numbers), n_configurations))

    for rand_iter in tqdm(range(n_configurations)):

        nodes = generate_network(n, r_max)
        A = adjacency_matrix(nodes, tau_max)
        At = disable_links(A,200000, delta_time, mode = mode)

        for i in range(len(packet_numbers)):

            #generating new packets every time since their number changes at every iteration
            packets = np.random.randint(1, n, (packet_numbers[i], 2))
            packets[:,1] = 0

            results = greedy_routing(At, packets, ttr, delta_time)
            arrival_times = np.array([packet.arrival_times[-1] for packet in results])
            outputs[i, rand_iter] = np.max(arrival_times)
            deviation_arr[i, rand_iter] = deviations(A, results)

    arrival_times = np.mean(outputs, axis = 1)

    axs[0].semilogx(packet_numbers, arrival_times, linestyle="-", c = color)
    axs[0].semilogx(packet_numbers, arrival_times - np.array(packet_numbers) * ttr, linestyle="-.", c=color)
    axs[1].semilogx(
        packet_numbers, 
        throughput(np.array(packet_numbers), arrival_times), 
        linestyle="--",
        c=color
    )
    axs[2].semilogx(packet_numbers, np.mean(deviation_arr, axis = 1), linestyle="--", c=color)

axs[0].semilogx(packet_numbers, np.array(packet_numbers) * ttr, linestyle = "--", c="black")
axs[0].set_ylabel("Time(s)")
axs[0].set_xlabel("#Packets")
axs[0].set_title("Mean arrival time vs mean idle time")
axs[0].legend([
    "Arrival time(light)", 
    "Idle time(light)",
    "Arrival time(unst)",
    "Idle time(unst)",
    "Total Transmission Time"
])


axs[1].set_title("Throughput of the system")
axs[1].set_ylabel("Normalized Throughput")
axs[1].set_xlabel("# Packets")
axs[1].legend([
    "Throughput(light)", 
    "Throughput(unstable)",
])

axs[2].set_title("Percentage of packets that followed unoptimal route")
axs[2].set_ylabel("Percentage")
axs[2].set_xlabel("# Packets")
axs[2].legend([
    "Percentage(light)",
    "Percentage(unstable)"
])

plt.tight_layout()

# An insight on the plots

1) With the first plot we are visualizing the arrival time, the idle time and the total transmission 
time as a function of the maximum radius under light and unstable mode.
We observe a very similar behaviour under light and unstable mode indicating that even if the load of the network is increased the arrival time is not really affected by the number of disabled links.
One interesting thing to notice is that as the number of packets increses so does the idle time, this was expected because increasing the number of packets obviously increases the chances that Earth spends some time without receiving data.


2) With the second image we're pointing out the throughput as a function of the number of packets; what we observe is that 
the behaviour under light and unstable mode is similar but of course light mode gives us a higher throughput.
Initially the throughput is low because, with a small amount of packets, the percentage of time in which the Earth is idle is very high since it takes time for the first packet to reach it.
  

3) In the last image we can observe the percentage of packets that followed unoptimal routes as a function of the number of packets. What we get for the two different modes is a similar behaviour. 
As the plot shows the more the packets, the higher the percentage of packets that follows
unoptimal routes. Anyway as we could expect the percentage is always higher with unstable mode.
For both modes this value tends to increase because with high load also the queuing time is high, meaning that the chance of a packet following an unoptimal path because of topological changes increases.

# Epidemic Algorithm
A key problem affecting DTNs is the fact that the topology of network changes a lot over time. In some cases, it is not possible to predict these network changes. For example in our case study, some celestial bodies could suddenly pass between the Earth and our satellites causing the link to fail and we cannot know if and when the link will become active again.<br> 
The epidemic algorithm suggests an approach that we can use to solve these types of problems,because it does not require a complete previous knowledge of the network topology. The algorithm works like an epidemic; if it is not possible to send packets to Earth, each node spreads the packets to all nearby nodes. So this procedure creates many copies of the same packet, in fact pure Epidemic routing has usually a high message delivery ratio. We can see below the main phases of the algorithm.<br>

   * 1) If the link is up the packets are sent from the node to the Earth, if the link is down the packets are sent to all nearby nodes that they can reach(the topology of network is locally fully-connected, but it is possible that some links are temporarily interrupted)<br><br>
   
   * 2) This procedure is repeated for all nodes over time, until all the packets have arrived to Earth. But if a node has already sent a copy of the packets to its neighbour, it doesn't need to send it back.<br><br>
   
   * 3) It is possible that the packet is discarded. This happens when: the packet is sent but during the propagation time the link breaks or when the packet arrives but there are some collisions on the receiver because other packets arrive in the same moment.  In both cases this is implemented by checking beforehand if it is possible to send the packet, if it is not then the packet is just not sent which is equivalent to discarding the packet midair. In case of collision one of the colliding packets is kept because by using CDMA (Code Division Multiple Access) it is actually possible to distinguish between the different signals even if there is interference.<br><br>
   
Using Epidemic algorithm, unless a node always remains isolated, we are sure that all packets arrive to Earth. The problem is that many copies of the same packet can create a congestion in the network and an increase of total energy consumption for data transmission.
In this algorithm we assume to know the topology of network, namely if the links are active or not, only in the current moment in which we are. We have no information about the future, so we cannot route efficiently the packets, like it happens in the algorithms above.

Obviously as the number of nodes and of packets increases the load of the network does too and it does exponentially, this means that in case of sensors networks, that contain many nodes, or other extreme cases this might not be the best algorithmic choice.
### Earth occupation
Like we did for the greedy algorithm we would like to find out when the Earth is occupied receiving useful data.
Unlike in the greedy case though here we need to take into consideration also duplicate packets as they can not be called useful.

In the following section the Earth occupation is represented, in particular colored slots correspond to the reception of a useful packet while black slots correspond to the reception of a duplicate or nothing at all.

In [ ]:
transmission_time = T_tx
slot_time = delta_time

#generating new packets with earth as a destination
packets = np.random.randint(1, n, (250, 2))
packets[:, 1] = 0

#generating the new configuration
At = disable_links(A, 100000, slot_time, mode="unstable")

#performing the routing algorithm
packet_arrive, packet_time, vulnerable, anim_packets = epidemic(
    At,
    transmission_time,
    slot_time,
    packets,
    return_anim=True
)

#filtering packets that arrived to earth and removing duplicates
packets_to_earth = remove_duplicates(list(filter(lambda x: x.route[0] == 0, anim_packets)))

plot_idle(packets_to_earth, transmission_time)

### Some insights on the plot

It is immediately possible to notice that, in contrast with the greedy algorithm, the idle time is dominant in the representation meaning that the algorithm is taking a lot more time to send all the packets that are needed.
A thing that can be noticed straight away is that most of the idle time is concentrated in final part of the transmission because a lot more duplicates are present in the network at that time.

Just this graph shows that the greedy algorithm is probabily more robust to extreme situations and probably performs better in all cases. This is obviously mainly due to the knowledge advantage that it has if compared with the epidemic algorithm.


## Metrics analysis
Also in this case different metrics values are displayed in order to try and understand how the algorithm behaves in different situations. All the computations were repeated for 10 different configurations and the results averaged out in order to reduce the noise and the algorithm was tested both with light and unstable mode.

### Effects of the number of packets
In the next section the dependence of the arrival time, throughput and packet fraction from the number of packets is investigated.
The packet fraction is a quantity that we defined that represents the ratio between the number of useful packets received by Earth and the total number of packets received.

This is an important metric for this algorithm because it approximately shows how much useless information reaches Earth.

In [ ]:
#generate figure
fig = plt.figure(figsize=(15, 7))
    
gs = fig.add_gridspec(1, 2)
axs = []
axs.append(fig.add_subplot(gs[0, 0]))
axs.append(fig.add_subplot(gs[0, 1]))

#seeing how the system behaves as number of packets increases
max_rand = 10

#plots and analysis with 10 nodes and r_max
for mode in ["light", "unstable"]:

    color = ["blue", "red"]
    color = color[mode == "light"]

    #number packet that we want to send to earth
    packet_numbers = [2**i for i in range(10)]

    #temp variables used to store data that will be used for the plots
    arrival_times = np.zeros((len(packet_numbers), n_configurations))
    packet_fraction = np.zeros((len(packet_numbers), n_configurations))

    for sample in tqdm(range(n_configurations)):

        #generating a new network configuration
        nodes = generate_network(n, r_max)
        A = adjacency_matrix(nodes, tau_max)
        At = disable_links(A, 100000, slot_time, mode = mode)

        for i in range(len(packet_numbers)):

            #generating new packets with earth as a destination
            packets = np.random.randint(1, n, (packet_numbers[i], 2))
            packets[:, 1] = 0

            #performing the routing algorithm
            packet_arrive, _, vulnerable = epidemic(At, transmission_time, slot_time, packets)
            arrival_times[i, sample] = vulnerable[0] + transmission_time
            packet_fraction[i, sample] = len(set(packet_arrive))/len(packet_arrive)

    axs[0].semilogx(packet_numbers, np.mean(arrival_times, axis = 1), c = color)
    axs[1].semilogx(packet_numbers, np.mean(packet_fraction, axis = 1), c = color, linestyle="--")

axs[0].semilogx(packet_numbers, np.array(packet_numbers)*transmission_time, linestyle="--")
axs[0].set_ylabel("Time(s)")
axs[0].set_xlabel("#Packets")
axs[0].set_title("Mean arrival time")
axs[0].legend([
    "Arrival time(light)",
    "Arrival time(unstable)",
    "Total transmission time"
])

axs[1].set_ylabel("packet fraction")
axs[1].set_xlabel("#Packets")
axs[1].set_title("Packet delivery ratio")
axs[1].legend([
    "Packet fraction(light)",
    "Packet fraction(unstable)"
])

1) We can see that the mean arrival time in both cases increases linearly (the x axis is in logarithmical scale) with the number of packets. Normally one would expect to see an exponential trend, what is probably preventing this to happen is the fact that if a node knows that it can send the packet directly to Earth it doesn't send copies around. Even if the number of disabled links is higher than in the unstable case we do not see the exponential trend because the number of nodes that effectively receive the packet is lower since most connections are disabled.
It is possible to see that with a bigger number of nodes the arrival time becomes exponential, the analysis was not presented in this notebook because it takes more than one hour just to run one simulation.
This though shows that this algorithm can actually be used if the number of nodes in the network is contained.
As expected the light mode curve is below the unstable one.

2)From the second plot we can see that the packet fraction value decreses as the number of packets increases. This was obvious because more packets means more copies in the network meaning more congestion and thus more useless packets reaching Earth.

### Effects of the transmission time
The following cell wants to show the dependence of the various metrics from the transmission time, since the outcome of these measurements were very noisy the number of configurations used was increased to 30.

In [ ]:
#generate figure
fig = plt.figure(figsize=(15, 7))
   
gs = fig.add_gridspec(1, 2)
axs = []
axs.append(fig.add_subplot(gs[0, 0]))
axs.append(fig.add_subplot(gs[0, 1]))

#seeing how the system behaves as the transmission time increases
n_configurations = 30

packet_numbers = 100

for mode in ["light", "unstable"]:

    color = ["blue", "red"]
    color = color[mode == "light"]

    transm_time = np.arange(1, 200, 20)
    arrival_times = np.zeros((len(transm_time), n_configurations))
    packet_fraction = np.zeros((len(transm_time), n_configurations))

    for sample in tqdm(range(n_configurations)):

        nodes = generate_network(n, r_max)
        A = adjacency_matrix(nodes, tau_max)
        At = disable_links(A, 100000, slot_time, mode = mode)

        for i in range(len(transm_time)):

            packets = np.random.randint(1, n, (packet_numbers, 2))
            packet_arrive, _, vulnerable = epidemic(At, transm_time[i], slot_time, packets)
            arrival_times[i, sample] = vulnerable[0] + transm_time[i]
            packet_fraction[i, sample] = len(set(packet_arrive))/len(packet_arrive)

    axs[0].plot(transm_time, np.mean(arrival_times, axis = 1), c = color)
    axs[0].set_ylabel("Time(s)")
    axs[0].set_xlabel("Transmission time")
    axs[0].set_title("Mean arrival time")
    axs[0].legend([
    "Arrival time(light)",
    "Arrival time(unstable)",
    ])

    axs[1].plot(transm_time, np.mean(packet_fraction, axis = 1), c = color, linestyle = "--")
    axs[1].set_ylabel("packet fraction")
    axs[1].set_xlabel("Transmission time")
    axs[1].set_title("Packet delivery ratio")
    axs[1].legend([
    "Packet fraction(light)",
    "Packet fraction(unstable)"
    ])

axs[0].plot(transm_time, np.array(transm_time*packet_numbers), c="black", linestyle="--")

From the plots it is possible to notice some very awkward behaviours, in particular it looks like that when the transmission time becomes comparable with the propagation time the number of deactivated links becomes ininfluent.
This can be seen in both plots as from a certain point the two lines tend to overlap which is an unexpected behaviour.
The second plot looks strange because the packet fraction increases after a certain point, this is probably due to the fact that with very high transmission rate it is very likely that a link is deactivated before the packet reaches its destination meaning that not too many packets are forwarded epidemically reducing the number of useless packets that reach Earth.

## Algorithm comparison
From the Earth occupation plots it is already clear that the the greedy algorithm is more performing. The opposite would have been quite strange since it relies on more information than the epidemic one.

It is still important to compare the two performances in case the same conditions are used for both, on the plots below the arrival times are confronted. Only the unstable mode was used and the performances were compared firstly as functions of the transmission times and later as functions of the number of packets.

In [ ]:
#generate figure
fig = plt.figure(figsize=(15, 7))
    
gs = fig.add_gridspec(1, 2)
axs = []
axs.append(fig.add_subplot(gs[0]))
axs.append(fig.add_subplot(gs[1]))
max_rand = 10
 
#different transmission times
transm_time = np.arange(1, 200, 20)

#number packet that we want to send to earth
packet_numbers = [2**i for i in range(10)]

#temp variables needed for plots
arrival_times = np.zeros((len(packet_numbers), max_rand))
outputs = np.zeros((len(packet_numbers), max_rand))
outputs_ttr = np.zeros((len(packet_numbers), max_rand))
arrival_times_ttr = np.zeros((len(packet_numbers), max_rand))

for sample in tqdm(range(max_rand)):

    #generating a new configuration
    nodes = generate_network(n, r_max)
    A = adjacency_matrix(nodes, tau_max)
    At = disable_links(A, 100000, slot_time, mode = "unstable")

    #getting the arrival times as the number of packets changes
    for i in range(len(packet_numbers)):

        packets = np.random.randint(1, n, (packet_numbers[i], 2))
        packets[:, 1] = 0

        #greedy routing results
        results = greedy_routing(At, packets, transmission_time, delta_time)

        #epidemic routing results
        packet_arrive, _, vulnerable = epidemic(At, transmission_time, slot_time, packets)

        arrival_times_greedy = np.array([packet.arrival_times[-1] for packet in results])
        outputs[i, sample] = np.max(arrival_times_greedy)
        arrival_times[i, sample] = vulnerable[0] + transmission_time

    #getting the arrival times as the transmission time changes
    for i in range(len(transm_time)):

        packets = np.random.randint(1, n, (100, 2))
        packets[:, 1] = 0

        #greedy routing results
        results = greedy_routing(At, packets, transm_time[i], delta_time)

        #epidemic routing results
        packet_arrive, _, vulnerable = epidemic(At, transm_time[i], slot_time, packets)

        arrival_times_greedy = np.array([packet.arrival_times[-1] for packet in results])
        outputs_ttr[i, sample] = np.max(arrival_times_greedy)
        arrival_times_ttr[i, sample] = vulnerable[0] + transm_time[i]

arrival_times_greedy = np.mean(outputs, axis = 1)
axs[0].semilogx(packet_numbers, np.mean(arrival_times, axis = 1), c= "blue")
axs[0].semilogx(packet_numbers, arrival_times_greedy, c="red")

axs[0].semilogx(packet_numbers, np.array(packet_numbers)*transmission_time, linestyle="--", color="green")
axs[0].set_ylabel("Time(s)")
axs[0].set_xlabel("#Packets")
axs[0].set_title("Mean arrival time")
axs[0].legend([
    "Arrival time(epidemic)",
    "Arrival time(greedy)",
    "Total transmission time"
])

arrival_times_greedy = np.mean(outputs_ttr, axis = 1)
axs[1].plot(transm_time, np.mean(arrival_times_ttr, axis = 1), color = "blue")
axs[1].plot(transm_time, arrival_times_greedy, color = "red")
axs[1].plot(transm_time, np.array(transm_time)*100,color="green" ,linestyle="--")
axs[1].set_ylabel("Time(s)")
axs[1].set_xlabel("Transmission time")
axs[1].set_title("Mean arrival time")
axs[1].legend([
    "Arrival time (epidemics)",
    "Arrival time (greedy)",
    "Total transmission time"
])

As we can observe the arrival time with the epidemic approach increases much faster than with the greedy one in both cases. This confirms our expectations, because since the beginning we knew that the greedy algorithm was the one with the strictest hypothesis but also the most efficient. 

In particular, as expected the epidemic algorithm performs really poorly in case many packets need to be forwarded because more packets generate a lot of duplicates this does not happen for the greedy algorithm so even if some packets might follow a better path (in the epidemic case), the extra load generated by the network is too much for the algorithm to compete with the greedy one.

# Conclusions

Here we want to sum up the results of our efforts by recapping which were the initial objectives and how the two 
different approaches tackled the problem. 

So our goal was to find an effective routing algorithm in the scenario of Delay Tolerant Networks, where nodes are 
located very far from each other and the links are not always working. 

The first strategy which we studied and implemented is a greedy algorithm which turned out to be very efficient and 
probably the best type of approach when dealing with DTNs.
We can say that this algorithm is very efficient because Earth (which is the destination node) is always busy in the
sense that there are almost no idle times for the destination node which is always receiving packets.


The two main reasons why we also tested the epidemic algorithm were

1) It needs much less information about the system to work properly.

2) We thought that because packets are sent everywhere some packets would arrive to Earth faster than they would with the greedy algorithm, this because the greedy algorithm does not keep queues in consideration when computing the best path meaning that some packets may be forced to take strange routes.
Unfortunately the amount of duplicate packets was more than enough to overcome this effect making the epidemic algorithm actually way worse.

In [1] the authors actually express very clearly the fact that in many cases the epidemic algorithm becomes unfeasible because it fills up the buffers and it generates too many packets.
From the simulations though we saw that for small networks where the topology can not be known in advance, this approach might actually work decently well as it takes all the packets to destination.

In [ ]:
## Animation: disabled links

%matplotlib notebook 

nodes = generate_network(n,r_max, seed = network_seed)

A = adjacency_matrix(nodes, tau_max)
At = disable_links(A,5000,1,mode="unstable")  #demonstration with unstable mode

n_updates = At.shape[2]
links = get_links(A,nodes)
active_links = [get_links(At[:,:,t],nodes) for t in range(n_updates)]

fig = plt.figure()
ax = p3.Axes3D(fig)

data = links
lines = [ax.plot(dat[0, :], dat[1, :], dat[2, :],alpha=0.5)[0] for dat in links]

# Setting the axes properties
ax.set_xlim3d([-4e10, 4e10])
ax.set_xlabel('X')

ax.set_ylim3d([-4e10, 4e10])
ax.set_ylabel('Y')

ax.set_zlim3d([-4e10,4e10])
ax.set_zlabel('Z')

title = ax.set_title("Time = ")
snodes = [100] + [50 for x in initial_nodes[1:]]
cnodes = ["green"] + ["black" for x in initial_nodes[1:]]
ax.scatter(initial_nodes[:,0],initial_nodes[:,1],initial_nodes[:,2],s=snodes,color=cnodes,alpha=0.8)

legend_lines = [Line2D([0], [0], color="black", lw=4),
                Line2D([0], [0], color="red", lw=4)]
ax.legend(legend_lines, ("Active links","Disabled links"))

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, n_updates, fargs=(data, lines,active_links,title),
                              interval=10, blit=False)

plt.show()

In [ ]:
packets = packets_greedy
A = initial_A
coord_nodes = np.copy(initial_nodes)

n_pkt = len(packets)
coord = coord_nodes[[pkt.source for pkt in packets]]
path = [[pkt.source] + pkt.route for pkt in packets]
arrival_times = [pkt.arrival_times for pkt in packets]
start_times = [[pkt.arrival_times[j] - A[path[i][j],path[i][j+1]] for j in range(len(pkt.route))] for i,pkt in enumerate(packets)]

delay = max([pkt.arrival_time for pkt in packets])

next_hop = coord_nodes[[x.route[0] for x in packets]]
diff = next_hop - coord
dist = np.linalg.norm(diff,axis=1)
times = dist/c
vel = (diff.T / times).T

#keep a fixed copy to be able to run the animation multiple times
fixed_coord = np.copy(coord)  
fixed_vel = np.copy(vel)

In [ ]:
## Animation: packets

%matplotlib notebook 

coord = np.copy(fixed_coord)##
vel = np.copy(fixed_vel)

step = np.zeros(n_pkt, dtype=int) #current step

done = [False for x in packets]

fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111, projection='3d')
title = ax.set_title('Time = ')

snodes = [300] + [110 for x in packets]
cnodes = ["black","red","orange","yellow","green","blue","magenta","blueviolet","grey","black"]

pkt_color = [cnodes[pkt.source] for pkt in packets]
graph = ax.scatter(coord[:,0],coord[:,1],coord[:,2],s=20,color=pkt_color,marker="s")

ax.scatter(coord_nodes[:,0],coord_nodes[:,1],coord_nodes[:,2],s=snodes,color=cnodes,alpha=1)

ani4 = animation.FuncAnimation(fig, update_graph, interval=10, blit=False,
                              fargs=(graph,coord,coord_nodes,start_times,arrival_times,step,n_pkt,vel,path,done,title,2))

plt.show()

# Bibliography

[1] Jain, Sushant, Kevin Fall, and Rabin Patra. "Routing in a delay tolerant network." Proceedings of the 2004 conference on Applications, technologies, architectures, and protocols for computer communications. 2004.<br>
[2] Evan P.C. Jone, Paul A.S. War. "Routing Strategies for Delay-Tolerant Networks."